In [44]:
import numpy as np
import os
import cv2
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [45]:
data_dir ="/home/local/ZOHOCORP/mukesh-pt6058/Desktop/Flowerclassification/ML/flowers/"

In [46]:
print(os.listdir("/home/local/ZOHOCORP/mukesh-pt6058/Desktop/Flowerclassification/ML/flowers/"))

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [47]:
batch_size = 32
img_height = 180
img_width = 180

In [48]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4323 files belonging to 5 classes.
Using 3459 files for training.


In [49]:
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4323 files belonging to 5 classes.
Using 864 files for validation.


In [50]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [51]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [52]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [53]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [54]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [55]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [68]:
epochs=35
model.fit(
  train_ds,
  validation_data=valid_ds,
  epochs=epochs
)

Epoch 1/35
109/109 [==============================] - 19s 178ms/step - loss: 0.2240 - accuracy: 0.9263 - val_loss: 1.3214 - val_accuracy: 0.7072
Epoch 2/35
109/109 [==============================] - 20s 179ms/step - loss: 0.1642 - accuracy: 0.9439 - val_loss: 1.3823 - val_accuracy: 0.6852
Epoch 3/35
109/109 [==============================] - 20s 179ms/step - loss: 0.2154 - accuracy: 0.9321 - val_loss: 1.2815 - val_accuracy: 0.7141
Epoch 4/35
109/109 [==============================] - 19s 178ms/step - loss: 0.1577 - accuracy: 0.9422 - val_loss: 1.3745 - val_accuracy: 0.7060
Epoch 5/35
109/109 [==============================] - 20s 178ms/step - loss: 0.1613 - accuracy: 0.9425 - val_loss: 1.3469 - val_accuracy: 0.7049
Epoch 6/35
109/109 [==============================] - 20s 178ms/step - loss: 0.1598 - accuracy: 0.9456 - val_loss: 1.3557 - val_accuracy: 0.7350
Epoch 7/35
109/109 [==============================] - 19s 178ms/step - loss: 0.1361 - accuracy: 0.9517 - val_loss: 1.3513 - val_ac

In [71]:

from keras.preprocessing import image
image = tf.keras.preprocessing.image.load_img('/home/local/ZOHOCORP/mukesh-pt6058/Desktop/Flowerclassification/ML/tulip2.jpeg', target_size=(180,180))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  
result = model.predict(input_arr)
prediction = np.argmax(result)

print(class_names[prediction])

1/1 [==============================] - 0s 28ms/step
tulip
